## Import Embedding Model

In [1]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("avsolatorio/GIST-small-Embedding-v0")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/68.0k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [12]:
model[0].auto_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [14]:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params}")

Trainable parameters: 33360000


## Download dataset

In [2]:
!kaggle datasets download coldn00ldes/sarcasm-corpus-v2oraby-et-al
!unzip sarcasm-corpus-v2oraby-et-al.zip

Dataset URL: https://www.kaggle.com/datasets/coldn00ldes/sarcasm-corpus-v2oraby-et-al
License(s): Attribution 4.0 International (CC BY 4.0)
  0% 0.00/1.04M [00:00<?, ?B/s]
100% 1.04M/1.04M [00:00<00:00, 84.9MB/s]
Archive:  sarcasm-corpus-v2oraby-et-al.zip
  inflating: GEN-sarc-notsarc.csv    
  inflating: HYP-sarc-notsarc.csv    
  inflating: RQ-sarc-notsarc.csv     


In [3]:
import pandas as pd

df = pd.read_csv("GEN-sarc-notsarc.csv")

class_mapping = {'notsarc': 0, 'sarc': 1}
df['class_encoded'] = df['class'].map(class_mapping)

#### Data splitting

In [4]:
from sklearn.model_selection import train_test_split

train_data, temp_data = train_test_split(df, test_size=0.2,  stratify=df['class_encoded'], random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=0.5, stratify=temp_data['class_encoded'], random_state=42)

#### CPU or Cuda

In [5]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("Device Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

device = 'cuda' if torch.cuda.is_available() else 'cpu'

CUDA Available: True
Device Name: Tesla T4


#### Import Libraries

In [7]:
import torch
from torch.utils.data import DataLoader
from torch import nn
from torch.optim import AdamW
from sklearn.metrics import precision_recall_fscore_support
from sentence_transformers import SentenceTransformer, InputExample
import os
import zipfile

In [8]:
def prepare_data(df, text_column, label_column):
    lst = []
    for _, row in df.iterrows():
        lst.append(InputExample(texts=[row[text_column]], label=row[label_column]))
    return lst

train_examples = prepare_data(train_data, text_column="text", label_column="class_encoded")
validation_examples = prepare_data(val_data, text_column="text", label_column="class_encoded")

In [9]:
def collate_fn(batch):
    texts = [example.texts[0] for example in batch]
    labels = [example.label for example in batch]

    tokenized = model.tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")
    labels = torch.tensor(labels, dtype=torch.float)

    return tokenized, labels

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16, collate_fn=collate_fn)
validation_dataloader = DataLoader(validation_examples, shuffle=False, batch_size=16, collate_fn=collate_fn)

In [20]:
class ClassificationModel(nn.Module):
    def __init__(self, base_model):
        super(ClassificationModel, self).__init__()
        self.base_model = base_model
        self.classifier = nn.Linear(base_model.get_sentence_embedding_dimension(), 1)

    def forward(self, tokenized_inputs):
        embeddings = self.base_model(tokenized_inputs)["sentence_embedding"]
        logits = self.classifier(embeddings)
        return logits

In [21]:
from peft import LoraConfig, get_peft_model

target_modules = ['query', 'key', 'value']  # 'dense'

lora_config = LoraConfig(
    task_type="SEQ_CLS",
    r=4,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=target_modules
)

for layer in model[0].auto_model.encoder.layer:
    layer = get_peft_model(layer, lora_config)

In [22]:
classification_model = ClassificationModel(model).to(device)

In [23]:
trainable_params = sum(p.numel() for p in classification_model.parameters() if p.requires_grad)
print(f"Trainable parameters: {trainable_params}")

Trainable parameters: 12177409


In [24]:
classification_model.base_model[0].auto_model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): lora.Linear(
              (base_layer): Linear(in_features=384, out_features=384, bias=True)
              (lora_dropout): ModuleDict(
                (default): Dropout(p=0.1, inplace=False)
              )
              (lora_A): ModuleDict(
                (default): Linear(in_features=384, out_features=4, bias=False)
              )
              (lora_B): ModuleDict(
                (default): Linear(in_features=4, out_features=384, bias=False)
              )
              (lora_embedding_A):

In [25]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = AdamW(filter(lambda p: p.requires_grad, classification_model.parameters()), lr=0.00001)

In [26]:
metrics_per_epoch = []
num_epochs = 20
checkpoint_dir = "model_checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)

for epoch in range(num_epochs):
    classification_model.train()
    all_train_predictions, all_train_labels = [], []
    train_loss = 0.0
    val_loss = 0.0

    for batch in train_dataloader:
        tokenized_inputs, labels = batch
        tokenized_inputs = {key: val.to(device) for key, val in tokenized_inputs.items()}
        labels = labels.to(device)

        optimizer.zero_grad()
        logits = classification_model(tokenized_inputs)
        loss = loss_fn(logits.squeeze(), labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        probabilities = torch.sigmoid(logits).squeeze()
        predictions = (probabilities > 0.5).long()

        all_train_predictions.extend(predictions.cpu().numpy())
        all_train_labels.extend(labels.cpu().numpy())

    train_loss /= len(train_dataloader)
    train_precision, train_recall, train_f1, train_support = precision_recall_fscore_support(
        all_train_labels, all_train_predictions, average=None, zero_division = 0
    )

    # Validation phase
    classification_model.eval()
    all_val_predictions, all_val_labels = [], []
    with torch.no_grad():
        for batch in validation_dataloader:
            tokenized_inputs, labels = batch
            tokenized_inputs = {key: val.to(device) for key, val in tokenized_inputs.items()}
            labels = labels.to(device)

            logits = classification_model(tokenized_inputs)
            loss = loss_fn(logits.squeeze(), labels)
            val_loss += loss.item()

            probabilities = torch.sigmoid(logits).squeeze()
            predictions = (probabilities > 0.5).long()

            all_val_predictions.extend(predictions.cpu().numpy())
            all_val_labels.extend(labels.cpu().numpy())

    val_loss /= len(validation_dataloader)
    val_precision, val_recall, val_f1, val_support = precision_recall_fscore_support(
        all_val_labels, all_val_predictions, average=None, zero_division=0
    )

    # Save metrics for this epoch
    epoch_metrics = {
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "val_loss": val_loss,
        "train_precision": train_precision,
        "train_recall": train_recall,
        "train_f1": train_f1,
        "val_precision": val_precision,
        "val_recall": val_recall,
        "val_f1": val_f1
    }
    metrics_per_epoch.append(epoch_metrics)

    base_model_path = os.path.join(checkpoint_dir, f"epoch{epoch + 1}_gist_model")
    classifier_path = os.path.join(checkpoint_dir, f"epoch{epoch + 1}_classifier_weights.pth")
    classification_model.base_model.save(base_model_path)
    torch.save(classification_model.classifier.state_dict(), classifier_path)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Training -> Loss: {train_loss}")
    print(f"  Precision: {train_precision}    Recall: {train_recall}    F1: {train_f1}")
    print(f"Validation -> Loss: {val_loss}")
    print(f"  Precision: {val_precision}    Recall: {val_recall}    F1: {val_f1}")
    print("============================================================================================================")

KeyboardInterrupt: 

In [ ]:
metrics_df = pd.DataFrame(metrics_per_epoch)
metrics_df.to_csv("epoch_metrics.csv", index=False)

In [ ]:
!zip -r /content/model_checkpoints/epoch8_gist_model.zip /content/model_checkpoints/epoch8_gist_model
!zip -r /content/model_checkpoints/epoch18_gist_model.zip /content/model_checkpoints/epoch18_gist_model
!zip -r /content/model_checkpoints/epoch20_gist_model.zip /content/model_checkpoints/epoch20_gist_model

  adding: content/model_checkpoints/epoch8_gist_model/ (stored 0%)
  adding: content/model_checkpoints/epoch8_gist_model/1_Pooling/ (stored 0%)
  adding: content/model_checkpoints/epoch8_gist_model/1_Pooling/config.json (deflated 57%)
  adding: content/model_checkpoints/epoch8_gist_model/tokenizer.json (deflated 71%)
  adding: content/model_checkpoints/epoch8_gist_model/config_sentence_transformers.json (deflated 34%)
  adding: content/model_checkpoints/epoch8_gist_model/modules.json (deflated 62%)
  adding: content/model_checkpoints/epoch8_gist_model/config.json (deflated 48%)
  adding: content/model_checkpoints/epoch8_gist_model/model.safetensors (deflated 8%)
  adding: content/model_checkpoints/epoch8_gist_model/vocab.txt (deflated 53%)
  adding: content/model_checkpoints/epoch8_gist_model/2_Normalize/ (stored 0%)
  adding: content/model_checkpoints/epoch8_gist_model/sentence_bert_config.json (deflated 4%)
  adding: content/model_checkpoints/epoch8_gist_model/special_tokens_map.json